In [1]:
#import libraries needed
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize

In [2]:
#read data with embeddings
datawemb = pd.read_csv("datawembeddings.csv")

In [3]:
datawemb.head()

,url,job_title,description,job_type,company,location,full_info_clean,duplicated,min_pay,max_pay,embeddings
0,https://www.mycareersfuture.gov.sg/job/custome...,PRODUCTION CONTROL MANAGER,JOB DESCIPTION\r\n\r\n planning and organisin...,"Permanent, Full Time",Snl Logistics Pte Ltd,31 GUL CIRCLE 629569,production control manager snl logistics pte l...,False,2000.0,3400.0,"[-0.1925869, 0.7930996, 0.8500719, 0.048986174..."
1,https://www.mycareersfuture.gov.sg/job/enginee...,Design Engineer ( Mechanical / Electrical),SUMMARY\r\n\r\n This position is respons...,Full Time,Jamco Aero Design &Amp; Engineering Private Li...,Singapore,design engineer mechanical electrical jamco ae...,False,2500.0,4500.0,"[-0.143587, 0.8263468, 0.8404796, -0.10907824,..."
2,https://www.mycareersfuture.gov.sg/job/sales/b...,Business Development Executive,Job description\r\nWho we are:\r\nWe are a log...,"Part Time, Permanent",Airpak Express Pte Ltd,"TECHPLAS INDUSTRIAL BUILDING, 45 CHANGI SOUTH ...",business development executive airpak express ...,False,3200.0,3500.0,"[-0.4457556, 0.78975, 1.2645174, -0.2784788, 0..."
3,https://www.mycareersfuture.gov.sg/job/banking...,Senior / Data Scientist,The ideal candidate should have a good underst...,"Permanent, Full Time",Singapore Exchange Limited,"SGX CENTRE I, 2 SHENTON WAY 068804",senior data scientist singapore exchange limit...,False,9000.0,14000.0,"[-0.80349284, 1.0287868, 1.3000816, 0.15158106..."
4,https://www.mycareersfuture.gov.sg/job/archite...,8890-Sales Consultant [ Digital Software| Saas...,Sales Consultant (Digital Software)\r\nLocatio...,"Permanent, Full Time",The Supreme Hr Advisory Pte. Ltd.,"SHENTON HOUSE, 3 SHENTON WAY 068805",8890 sales consultant digital software saas in...,False,3000.0,4500.0,"[-0.6867803, 1.2084837, 0.8840144, 0.000840738..."


In [4]:
numdescriptions = datawemb.shape[0]

In [5]:
#function to clean array which is read as strings
def clean(x):
    for i in range(len(x)):
        it = x[i]
        it = it.replace("[", "")
        it = it.replace(" ", "")
        it = it.replace("]", "")
        it = float(it)
        x[i] = it
    
    return np.array(x)

In [6]:
#convert emblist which is read as strings to array
datawemb['embeddings'] = datawemb['embeddings'].map(lambda x: x.split(","))
datawemb['embeddings'] = datawemb['embeddings'].map(lambda x: clean(x))

In [7]:
#function for data cleaning of inputs
stop_words = set(stopwords.words('english'))
def rm_stopwords(tokens):
    return [i for i in tokens if i not in stop_words and i]

In [8]:
#function for data cleaning of inputs
def process_input(inputd):
    inputd = " ".join(inputd )
    inputd = inputd.replace("\n"," ")
    inputd = inputd .replace("\r"," ")
    inputd = re.sub('[^A-Za-z0-9]+', ' ', inputd)
    inputd = inputd.lower()
    inputd = word_tokenize(inputd)
    inputd = rm_stopwords(inputd)
    inputd = " ".join(inputd)
    return inputd

In [9]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

C:\Users\65877\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
#recommendation function
def recommendation(input):
    pinput = process_input(input)
    inemb = sbert_model.encode(pinput)
    sim_scores = []
    for i in range(numdescriptions):
        score = cosine_similarity([inemb], [datawemb['embeddings'][i]])
        sim_scores.append((i, score))
    sim_scores.sort(key=lambda x: x[1])
    ind = list(zip(*sim_scores[-10:]))[0]
    return datawemb.iloc[list(ind)]

In [14]:
#testing 1
test_input1=["python","R","sql","git","flask","docker"]

recommendation(test_input1)

,url,job_title,description,job_type,company,location,full_info_clean,duplicated,min_pay,max_pay,embeddings
4473,https://sg.jobsdb.com/job/Data-Analyst-2d64ca7...,Data Analyst,Responsibilities: - \r\n1. Write software to...,Full time,Cyber Sense Technologies Pte. Ltd.,Singapore,data analyst cyber sense technologies pte ltd ...,False,NaN,NaN,"[-0.3170261, 1.2366338, 1.1658409, 0.12712215,..."
5672,https://sg.jobsdb.com/job/Senior-Software-Deve...,Senior Software Developer,"Task, Duties & Responsibilities:\r\n‚Ä¢ Workin...",Full time,ACHIEVE TECHNOLOGY ASIA PACIFIC PTE LTD,Eunos,senior software developer achieve technology a...,False,6500.0,9500.0,"[-0.32518852, 1.0980166, 0.78904027, -0.028768..."
5916,https://sg.jobsdb.com/job/Senior-Data-Analyst-...,"Senior Data Analyst, Advisory",Responsibility\r\n\r\nAbility to perform ident...,Full time,ST Engineering,Singapore,senior data analyst advisory st engineering fu...,False,NaN,NaN,"[-0.78059465, 1.3436424, 0.6724166, 0.5116054,..."
4947,https://sg.jobsdb.com/job/Data-Engineer-63fbe1...,Data Engineer,"Key Skills: Python, SQL, Unix, AWS\r\nObjectiv...",Permanent,UNISON CONSULTING PTE. LTD.,Changi,data engineer unison consulting pte ltd perman...,False,4000.0,5000.0,"[-0.23845917, 1.0551326, 0.79684275, 0.1497645..."
2667,https://sg.jobsdb.com/job/Technical-Specialist...,Technical Specialist / Consultant | Technology...,Job Description Summary\r\nDesign & Developmen...,Permanent,AIA Singapore,Singapore,technical specialist consultant technology spe...,False,6200.0,11500.0,"[-0.24554317, 1.2490017, 0.72966975, 0.3550323..."
2320,https://sg.jobsdb.com/job/Python-Developer-342...,Python Developer,\r\nExtensive experience in implementing scala...,Permanent,KG Information Systems Pte. Ltd.,Raffles Place,python developer kg information systems pte lt...,False,5000.0,8500.0,"[-0.42786327, 1.1450716, 0.968279, 0.17812085,..."
3197,https://sg.jobsdb.com/job/Senior-Data-Analyst-...,Senior Data Analyst (bank) (JT) (Machine Learn...,#NAME?,Contract,A-IT Software Services,Singapore,senior data analyst bank jt machine learning s...,False,6500.0,8000.0,"[-0.7132349, 1.1304373, 0.14359474, -0.0823287..."
3223,https://sg.jobsdb.com/job/Data-Engineer-0e8ed9...,Data Engineer,Job Title : Data Engineer\r\nJob description (...,Contract,U3 Infotech Pte Ltd,Singapore,data engineer u3 infotech pte ltd contract sin...,False,9000.0,14000.0,"[-0.2894813, 1.2983376, 0.6473273, 0.18196937,..."
3501,https://sg.jobsdb.com/job/Data-Ops-Engineer-33...,Data Ops Engineer 007,• You have experience building production-grad...,Contract,U3 Infotech Pte Ltd,Singapore,data ops engineer 007 u3 infotech pte ltd cont...,False,7000.0,9500.0,"[-0.15325922, 1.032224, 0.82651114, 0.21775155..."
4302,https://sg.jobsdb.com/job/Data-Engineer-b5b53d...,Data Engineer,Job Title : Data Engineer\r\nJob description (...,Contract,U3 Infotech Pte Ltd,Singapore,data engineer u3 infotech pte ltd contract sin...,False,6000.0,9000.0,"[-0.28830102, 1.3850135, 0.55168843, 0.2451738..."


In [15]:
#testing 2
test_input2=['engineer', 'cloud', 'architecture', 'operating system', 'sql']

recommendation(test_input2)

,url,job_title,description,job_type,company,location,full_info_clean,duplicated,min_pay,max_pay,embeddings
2444,https://sg.jobsdb.com/job/Database-Administrat...,Database Administrator,This is predominantly Senior Engineer role wor...,Full time,REC Solar,Tuas,database administrator rec solar full time tua...,False,NaN,NaN,"[-0.48589498, 1.1524243, 1.185113, 0.105670854..."
5916,https://sg.jobsdb.com/job/Senior-Data-Analyst-...,"Senior Data Analyst, Advisory",Responsibility\r\n\r\nAbility to perform ident...,Full time,ST Engineering,Singapore,senior data analyst advisory st engineering fu...,False,NaN,NaN,"[-0.78059465, 1.3436424, 0.6724166, 0.5116054,..."
3223,https://sg.jobsdb.com/job/Data-Engineer-0e8ed9...,Data Engineer,Job Title : Data Engineer\r\nJob description (...,Contract,U3 Infotech Pte Ltd,Singapore,data engineer u3 infotech pte ltd contract sin...,False,9000.0,14000.0,"[-0.2894813, 1.2983376, 0.6473273, 0.18196937,..."
4302,https://sg.jobsdb.com/job/Data-Engineer-b5b53d...,Data Engineer,Job Title : Data Engineer\r\nJob description (...,Contract,U3 Infotech Pte Ltd,Singapore,data engineer u3 infotech pte ltd contract sin...,False,6000.0,9000.0,"[-0.28830102, 1.3850135, 0.55168843, 0.2451738..."
3181,https://sg.jobsdb.com/job/AWS-Data-Engineer-bc...,AWS Data Engineer,Required Skills:\r\n* AWS Data Engineers\r\n* ...,Contract,IDC Technologies,Central Singapore,aws data engineer idc technologies contract ce...,False,6000.0,7500.0,"[-0.3853913, 0.99493074, 1.3304172, 0.53653353..."
5818,https://sg.jobsdb.com/job/Database-Administrat...,Senior/Database Administrator,Responsibilities: \r\n\r\nResponsible for impl...,Full time,CVista HR Consulting Pte Ltd,Tuas,senior database administrator cvista hr consul...,False,6500.0,6500.0,"[-0.39711425, 1.060669, 1.420305, 0.48891968, ..."
3197,https://sg.jobsdb.com/job/Senior-Data-Analyst-...,Senior Data Analyst (bank) (JT) (Machine Learn...,#NAME?,Contract,A-IT Software Services,Singapore,senior data analyst bank jt machine learning s...,False,6500.0,8000.0,"[-0.7132349, 1.1304373, 0.14359474, -0.0823287..."
4589,https://sg.jobsdb.com/job/Engineer-516beefba9a...,Engineer (Database Administrator),This role work under Production System & Devel...,Full time,RecruitPlus Consulting,Tuas,engineer database administrator recruitplus co...,False,NaN,NaN,"[-0.44864976, 1.1188912, 1.5694858, 0.30545726..."
3501,https://sg.jobsdb.com/job/Data-Ops-Engineer-33...,Data Ops Engineer 007,• You have experience building production-grad...,Contract,U3 Infotech Pte Ltd,Singapore,data ops engineer 007 u3 infotech pte ltd cont...,False,7000.0,9500.0,"[-0.15325922, 1.032224, 0.82651114, 0.21775155..."
2667,https://sg.jobsdb.com/job/Technical-Specialist...,Technical Specialist / Consultant | Technology...,Job Description Summary\r\nDesign & Developmen...,Permanent,AIA Singapore,Singapore,technical specialist consultant technology spe...,False,6200.0,11500.0,"[-0.24554317, 1.2490017, 0.72966975, 0.3550323..."
